Explanation

	1.	auc_brier_ece Function: This function calculates the combined score using AUC, Brier Score, and Expected Calibration Error (ECE).
	2.	evaluate_submission Function: This function loads the ground truth (answer.csv) and submission (submission.csv) files, computes the evaluation score using auc_brier_ece, and prints the score.
	3.	main_evaluation Function: This function specifies the paths for the ground truth and submission files, calls evaluate_submission, and prints the final evaluation score.
	4.	Call main_evaluation: At the end of the script, main_evaluation is called to execute the evaluation.

How to Use

	1.	Train the Model: Run the cells containing the training loop (main function).
	2.	Generate the Submission File: Run the cells containing the submission creation functions (create_submission function).
	3.	Evaluate the Submission: Run the cells containing the evaluation code (main_evaluation function).

In [1]:
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from sklearn.calibration import calibration_curve
from sklearn.metrics import mean_squared_error, roc_auc_score

# Configuration
class Config:
    SR = 32000  # Sampling rate
    N_MELS = 128  # Number of Mel bands
    N_MFCC = 13
    MAX_SEQ_LEN = 200  # Maximum sequence length for input to the model
    ROOT_FOLDER = '/content/drive/MyDrive/dataset/TeamDeepwave/dataset/open/'  # Dataset root folder
    SUBSET_SIZE = 50000  # Size of the subset to use
    OUTPUT_FOLDER = '/content/drive/MyDrive/dataset/TeamDeepwave/dataset/preprocessed/'  # Folder to save PNG images
    BATCH_SIZE = 64
    N_EPOCHS = 10
    LR = 1e-4

CONFIG = Config()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Ensure output directories exist
os.makedirs(os.path.join(CONFIG.OUTPUT_FOLDER, 'train', 'real'), exist_ok=True)
os.makedirs(os.path.join(CONFIG.OUTPUT_FOLDER, 'train', 'fake'), exist_ok=True)
os.makedirs(os.path.join(CONFIG.OUTPUT_FOLDER, 'test'), exist_ok=True)

# Model Definitions
class BiLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # hidden_dim * 2 because it's bidirectional
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        num_directions = 2 if self.lstm.bidirectional else 1
        h_0 = torch.zeros(self.lstm.num_layers * num_directions, x.size(0), self.lstm.hidden_size).to(device)
        c_0 = torch.zeros(self.lstm.num_layers * num_directions, x.size(0), self.lstm.hidden_size).to(device)

        x = self.dropout(x)
        lstm_out, _ = self.lstm(x, (h_0, c_0))
        x = self.fc(self.dropout(lstm_out[:, -1, :]))
        return x

class CNN(nn.Module):
    def __init__(self, output_dim):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.gelu(self.conv1(x)))  # Replace F.relu with F.gelu
        x = self.pool(F.gelu(self.conv2(x)))  # Replace F.relu with F.gelu
        x = x.view(-1, 64 * 32 * 32)
        x = F.gelu(self.fc1(x))  # Replace F.relu with F.gelu
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class CombinedModel(nn.Module):
    def __init__(self, lstm_input_dim, lstm_hidden_dim, lstm_output_dim, lstm_n_layers, lstm_bidirectional, lstm_dropout, cnn_output_dim):
        super(CombinedModel, self).__init__()
        self.lstm = BiLSTM(lstm_input_dim, lstm_hidden_dim, lstm_output_dim, lstm_n_layers, lstm_bidirectional, lstm_dropout)
        self.cnn = CNN(cnn_output_dim)
        self.fc = nn.Linear(lstm_output_dim + cnn_output_dim, 2)

    def forward(self, mfcc, mel):
        lstm_out = torch.zeros(mfcc.size(0), 128).to(device)  # Dummy LSTM output if no MFCC is provided
        if mfcc is not None:
            mfcc = mfcc.permute(0, 2, 1)  # Change from (batch, channels, seq_len) to (batch, seq_len, input_dim)
            lstm_out = self.lstm(mfcc)
        cnn_out = self.cnn(mel)
        combined = torch.cat((lstm_out, cnn_out), dim=1)
        out = self.fc(combined)
        return out

# Data Loading and Preprocessing
def load_train_file_paths_and_labels(csv_path, subset_size=CONFIG.SUBSET_SIZE):
    df = pd.read_csv(csv_path)
    print("CSV Columns:", df.columns)  # Debug print to check columns
    print(df.head())  # Print first few rows to check structure
    df_subset = df.sample(n=subset_size)
    file_paths = df_subset['path'].apply(lambda x: os.path.join(CONFIG.ROOT_FOLDER, x)).tolist()
    labels = df_subset['label'].tolist()
    return file_paths, labels

def load_test_file_paths(csv_path, subset_size=CONFIG.SUBSET_SIZE):
    df = pd.read_csv(csv_path)
    print("CSV Columns:", df.columns)  # Debug print to check columns
    print(df.head())  # Print first few rows to check structure
    df_subset = df.sample(n=subset_size)
    file_paths = df_subset['path'].apply(lambda x: os.path.join(CONFIG.ROOT_FOLDER, x)).tolist()
    return file_paths

def save_mel_spectrogram(file_path, label, output_folder, is_test=False):
    try:
        print(f"Processing file: {file_path}")  # Debug print
        y, sr = librosa.load(file_path, sr=CONFIG.SR)
        mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=CONFIG.N_MELS)
        mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

        plt.figure(figsize=(10, 4))
        librosa.display.specshow(mel_spectrogram_db, sr=sr, x_axis='time', y_axis='mel')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Mel-spectrogram')
        plt.tight_layout()

        filename = os.path.basename(file_path).replace('.ogg', '.png')
        if is_test:
            output_path = os.path.join(output_folder, filename)
        else:
            if label == 'fake':
                output_path = os.path.join(output_folder, 'fake', filename)
            else:
                output_path = os.path.join(output_folder, 'real', filename)

        # Ensure the directory exists
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        plt.savefig(output_path)
        plt.close()
        print(f"Saved file: {output_path}")  # Debug print

    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

train_files, train_labels = load_train_file_paths_and_labels('/content/drive/MyDrive/dataset/TeamDeepwave/dataset/open/train.csv')
test_files = load_test_file_paths('/content/drive/MyDrive/dataset/TeamDeepwave/dataset/open/test.csv')

# Save Mel-spectrograms as PNG images for training data
for file_path, label in tqdm(zip(train_files, train_labels), total=len(train_files)):
    save_mel_spectrogram(file_path, label, os.path.join(CONFIG.OUTPUT_FOLDER, 'train'))

# Save Mel-spectrograms as PNG images for testing data
for file_path in tqdm(test_files, total=len(test_files)):
    save_mel_spectrogram(file_path, None, os.path.join(CONFIG.OUTPUT_FOLDER, 'test'), is_test=True)

print('Mel-spectrogram images saved successfully.')

CSV Columns: Index(['id', 'path', 'label'], dtype='object')
         id                  path label
0  RUNQPNJF  ./train/RUNQPNJF.ogg  real
1  JFAWUOGJ  ./train/JFAWUOGJ.ogg  fake
2  RDKEKEVX  ./train/RDKEKEVX.ogg  real
3  QYHJDOFK  ./train/QYHJDOFK.ogg  real
4  RSPQNHAO  ./train/RSPQNHAO.ogg  real
CSV Columns: Index(['id', 'path'], dtype='object')
           id                   path
0  TEST_00000  ./test/TEST_00000.ogg
1  TEST_00001  ./test/TEST_00001.ogg
2  TEST_00002  ./test/TEST_00002.ogg
3  TEST_00003  ./test/TEST_00003.ogg
4  TEST_00004  ./test/TEST_00004.ogg


  0%|          | 0/50000 [00:00<?, ?it/s]

Processing file: /content/drive/MyDrive/dataset/TeamDeepwave/dataset/open/./train/TQQDZACA.ogg


  0%|          | 1/50000 [00:21<296:35:23, 21.35s/it]

Saved file: /content/drive/MyDrive/dataset/TeamDeepwave/dataset/preprocessed/train/real/TQQDZACA.png
Processing file: /content/drive/MyDrive/dataset/TeamDeepwave/dataset/open/./train/NFPFAFHK.ogg


...

  3%|▎         | 1642/50000 [20:53<9:54:58,  1.35it/s] 

Saved file: /content/drive/MyDrive/dataset/TeamDeepwave/dataset/preprocessed/train/fake/XQKDDRXF.png
Processing file: /content/drive/MyDrive/dataset/TeamDeepwave/dataset/open/./train/WPBXIMWX.ogg


  3%|▎         | 1642/50000 [20:54<10:15:36,  1.31it/s]


KeyboardInterrupt: 

<Figure size 1000x400 with 0 Axes>

In [ ]:
# Dataset Class for Model Training
class CustomDataset(Dataset):
    def __init__(self, mfcc_files, mel_files, labels=None, transform=None):
        self.mfcc_files = mfcc_files
        self.mel_files = mel_files
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.mel_files)

    def __getitem__(self, idx):
        mel_image = Image.open(self.mel_files[idx]).convert('L')  # Convert to grayscale
        if self.transform:
            mel_image = self.transform(mel_image)

        if self.labels is not None:
            label = torch.tensor(self.labels[idx], dtype=torch.float32)
            mfcc = torch.zeros((CONFIG.N_MFCC, CONFIG.MAX_SEQ_LEN))
            return mfcc, mel_image, label
        return mel_image

# Training and validation functions
def train(model, loader, criterion, optimizer, device):
    model.train()
    epoch_loss = 0
    correct = 0
    total = 0
    for mfcc, mel, labels in loader:
        mfcc, mel, labels = mfcc.to(device), mel.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(mfcc, mel)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        _, labels = torch.max(labels.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = correct / total if total > 0 else 0  # Avoid division by zero
    return epoch_loss / len(loader) if len(loader) > 0 else 0, accuracy

def evaluate(model, loader, criterion, device):
    model.eval()
    epoch_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for mfcc, mel, labels in loader:
            mfcc, mel, labels = mfcc.to(device), mel.to(device), labels.to(device)
            outputs = model(mfcc, mel)
            loss = criterion(outputs, labels)
            epoch_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total if total > 0 else 0  # Avoid division by zero
    return epoch_loss / len(loader) if len(loader) > 0 else 0, accuracy

# Main training loop
def main():
    # Load train data
    train_files, train_labels = load_train_file_paths_and_labels('/content/drive/MyDrive/dataset/TeamDeepwave/dataset/open/train.csv')

    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])  # Grayscale normalization
    ])

    train_dataset = CustomDataset(train_files, train_files, train_labels, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True)

    # Model initialization
    model = CombinedModel(
        lstm_input_dim=CONFIG.N_MFCC,
        lstm_hidden_dim=128,
        lstm_output_dim=128,
        lstm_n_layers=2,
        lstm_bidirectional=True,
        lstm_dropout=0.5,
        cnn_output_dim=128
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=CONFIG.LR)

    # Training loop
    best_valid_loss = float('inf')
    for epoch in range(CONFIG.N_EPOCHS):
        train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
        # Assuming you have validation data and loader (val_loader)
        # valid_loss, valid_acc = evaluate(model, val_loader, criterion, device)

        # if valid_loss < best_valid_loss:
        #     best_valid_loss = valid_loss
        #     torch.save(model.state_dict(), 'best-model.pt')

        print(f'Epoch {epoch+1}')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        # print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

    # Save the model weights in .h5 format
    model_path = 'model_weights.h5'
    torch.save(model.state_dict(), model_path)
    print(f'Model weights saved to {model_path}')

if __name__ == "__main__":
    main()

# Submission creation functions
class TestDataset(Dataset):
    def __init__(self, mel_files, transform=None):
        self.mel_files = mel_files
        self.transform = transform

    def __len__(self):
        return len(self.mel_files)

    def __getitem__(self, idx):
        mel_image = Image.open(self.mel_files[idx]).convert('L')  # Convert to grayscale
        if self.transform:
            mel_image = self.transform(mel_image)
        return mel_image

def load_model(model_path):
    model = CombinedModel(
        lstm_input_dim=CONFIG.N_MFCC,
        lstm_hidden_dim=128,
        lstm_output_dim=128,
        lstm_n_layers=2,
        lstm_bidirectional=True,
        lstm_dropout=0.5,
        cnn_output_dim=128
    ).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

def predict_probabilities(model, data_loader, device='cpu'):
    all_probs = []
    with torch.no_grad():
        for mel in data_loader:
            mel = mel.to(device)
            outputs = model(None, mel)
            prob = torch.nn.functional.softmax(outputs, dim=1)
            all_probs.append(prob.cpu().numpy())
    return np.concatenate(all_probs)

def generate_submission_file(test_files, predicted_probabilities, output_file='submission.csv'):
    test_ids = [os.path.basename(f).replace('.ogg', '') for f in test_files]
    submission_df = pd.DataFrame(predicted_probabilities, columns=['fake', 'real'])
    submission_df.insert(0, 'id', test_ids)
    submission_df.to_csv(output_file, index=False)
    print(f'Submission file created successfully: {output_file}')

def create_submission():
    model_path = 'model_weights.h5'
    test_csv = '/content/drive/MyDrive/dataset/TeamDeepwave/dataset/open/test.csv'
    output_file = 'submission.csv'

    # Load test file paths
    df = pd.read_csv(test_csv)
    test_files = df['path'].apply(lambda x: os.path.join(CONFIG.ROOT_FOLDER, x)).tolist()

    # Preprocess test data
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])  # Grayscale normalization
    ])

    test_dataset = TestDataset(test_files, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)

    # Load model
    model = load_model(model_path)

    # Predict probabilities
    predicted_probabilities = predict_probabilities(model, test_loader, device=device)

    # Generate submission file
    generate_submission_file(test_files, predicted_probabilities, output_file)

# Generate the submission file
create_submission()

# Official Evaluation Code
def expected_calibration_error(y_true, y_prob, n_bins=10):
    prob_true, prob_pred = calibration_curve(y_true, y_prob, n_bins=n_bins, strategy='uniform')
    bin_totals = np.histogram(y_prob, bins=np.linspace(0, 1, n_bins + 1), density=False)[0]
    non_empty_bins = bin_totals > 0
    bin_weights = bin_totals / len(y_prob)
    bin_weights = bin_weights[non_empty_bins]
    prob_true = prob_true[:len(bin_weights)]
    prob_pred = prob_pred[:len(bin_weights)]
    ece = np.sum(bin_weights * np.abs(prob_true - prob_pred))
    return ece

def auc_brier_ece(answer_df, submission_df):
    # Check for missing values in submission_df
    if submission_df.isnull().values.any():
        raise ValueError("The submission dataframe contains missing values.")

    # Check if the number and names of columns are the same in both dataframes
    if len(answer_df.columns) != len(submission_df.columns) or not all(answer_df.columns == submission_df.columns):
        raise ValueError("The columns of the answer and submission dataframes do not match.")

    submission_df = submission_df[submission_df.iloc[:, 0].isin(answer_df.iloc[:, 0])]
    submission_df.index = range(submission_df.shape[0])

    # Calculate AUC for each class
    auc_scores = []
    for column in answer_df.columns[1:]:
        y_true = answer_df[column]
        y_scores = submission_df[column]
        auc = roc_auc_score(y_true, y_scores)
        auc_scores.append(auc)

    # Calculate mean AUC
    mean_auc = np.mean(auc_scores)

    brier_scores = []
    ece_scores = []

    # Calculate Brier Score and ECE for each class
    for column in answer_df.columns[1:]:
        y_true = answer_df[column].values
        y_prob = submission_df[column].values

        # Brier Score
        brier = mean_squared_error(y_true, y_prob)
        brier_scores.append(brier)

        # ECE
        ece = expected_calibration_error(y_true, y_prob)
        ece_scores.append(ece)

    # Calculate mean Brier Score and mean ECE
    mean_brier = np.mean(brier_scores)
    mean_ece = np.mean(ece_scores)

    # Calculate combined score
    combined_score = 0.5 * (1 - mean_auc) + 0.25 * mean_brier + 0.25 * mean_ece

    return combined_score

# Function to evaluate the model using the official evaluation metric
def evaluate_submission(answer_csv, submission_csv):
    answer_df = pd.read_csv(answer_csv)
    submission_df = pd.read_csv(submission_csv)
    score = auc_brier_ece(answer_df, submission_df)
    print(f'Evaluation Score: {score:.4f}')
    return score

def main_evaluation():
    answer_csv = '/content/drive/MyDrive/dataset/TeamDeepwave/dataset/open/answer.csv'
    submission_csv = 'submission.csv'
    score = evaluate_submission(answer_csv, submission_csv)
    print(f'Final Evaluation Score: {score:.4f}')

# Call the main_evaluation function to evaluate the submission
main_evaluation()


NameError: name 'load_train_file_paths_and_labels' is not defined